## COMP47670 - Assignment 2
**Student Name: Meleesha Mayola Dsouza, Nikil Mohan**<br>
**Student Number: 18200024, 18200037**

In [1]:
# Import the required libraries
import os
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Selecting review categories

For the purpose of this assignment we have chosen the following categories: 
1. Automotive
2. Health and Medical

In [6]:
endpoint = 'http://mlg.ucd.ie/modules/yalp/'
categories = ['health_medical_list.html', 'automotive_list.html'] 

def extract_page(url):
    response = requests.get(url)
    data = response.text

    soup = BeautifulSoup(data,'html.parser')
    return soup

def extract_reviews(url):
    soup = extract_page(url)

    reviews_block = soup.findAll("div", { "class" : "review" })
    review_list =[]
    for reviews in reviews_block:
        review = {}
        star = reviews.find('img')
        review["comments"] = reviews.find("p", { "class" : "text" }).get_text()
        review["rating"] = 'positive' if int(star.get('alt').split('-')[0]) >= 4 else 'negative'
        review_list.append(review)
        
    return review_list

def extract_data(url):
    soup = extract_page(url)

    links = soup.find_all('a')
    review_list =[]
    for link in links:
        url = endpoint + link.get('href')
        review = extract_reviews(url)
        review_list = review_list+review #concatenate the reviews from all the links
    return review_list
        
def create_database(category_name, reviews):
    header = reviews[0].keys()
    with open(category_name, 'w',newline='', encoding="utf-8") as output_file:
        dict_writer = csv.DictWriter(output_file, header)
        dict_writer.writeheader()
        dict_writer.writerows(reviews)


Create database for category A, Health and medical reviews

In [7]:
# extract Health and medical reviews
url = endpoint + categories[0]
review_list = extract_data(url)
create_database('Health_Medical.csv', review_list)

Create database for category B, Automotive reviews

In [8]:
# extract Automotive reviews
url = endpoint + categories[1]
review_list = extract_data(url)
create_database('Automotive.csv', review_list)

In [10]:
dataset_A = pd.read_csv('Health_Medical.csv')
print(dataset_A.shape)
dataset_B = pd.read_csv('Automotive.csv')
print(dataset_B.shape)

(1450, 2)
(1455, 2)


In [11]:
dataset_A.head()

,comments,rating
0,I have so many good things to say about this p...,positive
1,I found them to be highly skilled and an exper...,positive
2,Where do I even begin? This office has been so...,positive
3,I went in because I had toothache and needed a...,positive
4,Found a new dental office. This place is amazi...,positive


In [40]:
def preprocess_text(text):
    # this function takes a text as an input 
    # tokenizes the text into individual tokens
    tokenized_words = word_tokenize(text)
    # removes the punctuation
    normalised_words = [word for word in tokenized_words if word.isalpha()]
    
    # removes all the stop words
    return ' '.join([word for word in normalised_words if word.lower() not in stopwords.words('english')])
    

TASK 2 cross validation

task 3 hold out testing , i.e. train on data from one category and test on data from another.

In [44]:
def preprocess_reviews(dataset):
    for indx, comments in enumerate(dataset['comments']):
        dataset['comments'][indx] = preprocess_text(comments)

In [45]:
preprocess_reviews(dataset_A)

In [46]:
dataset_A.head()

,comments,rating
0,many good things say place needed cleaning fil...,positive
1,found highly skilled experienced dentistry wil...,positive
2,even begin office incredibly good family love ...,positive
3,went toothache needed cleaning really friendly...,positive
4,Found new dental office place amazing people w...,positive


In [47]:
preprocess_reviews(dataset_B)

In [49]:
dataset_B.head()

,comments,rating
0,arrived PM dealership closed thought left plen...,positive
1,dropped car Wednesday morning diagnostic John ...,negative
2,parents buying cars Donna Dunnivan decade alwa...,positive
3,recently bought another car Donna Dunnivan Coc...,positive
4,schedule appointment due airbag recall used we...,positive
